In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import MobileNetV2

# Base network to be shared (reused)
def create_base_network(input_shape):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, pooling='avg')
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = layers.Dense(128, activation='relu')(x)
    return models.Model(base_model.input, x)

# Create pairs for training
def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.'''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(10)]) - 1
    for d in range(10):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, 10)
            dn = (d + inc) % 10
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

input_shape = (224, 224, 3) # Example input shape
base_network = create_base_network(input_shape)

# Create the left and right inputs for the pairs
input_a = layers.Input(shape=input_shape)
input_b = layers.Input(shape=input_shape)

# Because we re-use the same instance `base_network`,
# the weights of the network will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

# Example distance layer (L1 distance)
distance = layers.Lambda(lambda embeddings: tf.abs(embeddings[0] - embeddings[1]))([processed_a, processed_b])

# Add a dense layer with a sigmoid unit to generate the similarity score
prediction = layers.Dense(1, activation='sigmoid')(distance)

# Connect the inputs with the outputs
model = models.Model(inputs=[input_a, input_b], outputs=prediction)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(), metrics=['accuracy'])

# Assume `pairs` and `labels` are prepared datasets
# pairs = [[imgA1, imgB1], [imgA2, imgB2], ...]
# labels = [1, 0, ...] where 1 is same class, 0 is different class
# train_model(model, pairs, labels) # Example function to train the model